# 04 - Pipeline Final para Submissão

**🎯 PROPÓSITO DESTE NOTEBOOK:**
Este notebook implementa o pipeline final para gerar as previsões de vendas para janeiro de 2023. Utiliza todos os dados de 2022 para treinar o modelo final e gera os arquivos de submissão nos formatos requeridos.

**🚀 ESTRATÉGIA OTIMIZADA:**
- **Treino com Dataset Completo**: Usa todos os dados de 2022 (sem divisão treino/validação)
- **Modelo Final**: LightGBM treinado com número ótimo de iterações
- **Predições Janeiro/2023**: Gera features para as 5 semanas de janeiro de 2023
- **Submissão**: Salva nos formatos CSV e Parquet na pasta correta

---

## Fluxo do Pipeline:
1. **Carregamento**: Dados processados com features de `02-Feature-Engineering-Dask.ipynb`
2. **Preparação**: Otimização de memória e uso do dataset completo de 2022
3. **Treinamento**: Modelo final LightGBM com dados completos
4. **Predição**: Geração de features para janeiro/2023 e previsões finais
5. **Submissão**: Arquivos finais em CSV e Parquet prontos para envio

In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Advanced ML
import lightgbm as lgb
from lightgbm.callback import early_stopping  # CORREÇÃO: Importar early_stopping
import xgboost as xgb

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time series
from datetime import datetime, timedelta
import gc

plt.style.use('default')
sns.set_palette("husl")

print('📚 Bibliotecas carregadas com sucesso!')
print('🎯 Iniciando fase de Modelagem e Treinamento')

📚 Bibliotecas carregadas com sucesso!
🎯 Iniciando fase de Modelagem e Treinamento


## 1. Carregamento dos Dados Processados

In [2]:
# Carregar dados com features processadas
print('📂 Carregando dados processados...')

# Verificar se os arquivos essenciais existem
import os
required_files = [
    '../data/dados_features_completo.parquet',  # Usar parquet (mais rápido)
    '../data/feature_engineering_metadata.pkl'
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print('❌ Arquivos não encontrados:')
    for f in missing_files:
        print(f'   • {f}')
    print('\n🔄 Execute primeiro o notebook 02-Feature-Engineering-Dask.ipynb')
else:
    print('✅ Todos os arquivos necessários encontrados')
    
    # Carregar dados principais (usar parquet para velocidade)
    print('📊 Carregando dataset (parquet)...')
    dados = pd.read_parquet('../data/dados_features_completo.parquet')
    
    # Carregar metadados
    with open('../data/feature_engineering_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    
    print(f'\n📊 Dados carregados com sucesso:')
    print(f'   • Shape: {dados.shape}')
    print(f'   • Período: {dados["semana"].min()} até {dados["semana"].max()}')
    print(f'   • Features disponíveis: {len(dados.columns)}')
    print(f'   • Memória: {dados.memory_usage(deep=True).sum() / (1024**2):.1f} MB')
    print(f'   • Estratégia: {metadata.get("estrategia", "Grid Inteligente")}')
    
    print(f'\n🔍 Metadados do processamento:')
    for key, value in metadata.items():
        if key not in ['features_criadas', 'data_processamento']:  # Skip long items
            print(f'   • {key}: {value}')
    
    print(f'\n✅ Pronto para modelagem!')

📂 Carregando dados processados...
✅ Todos os arquivos necessários encontrados
📊 Carregando dataset (parquet)...

📊 Dados carregados com sucesso:
   • Shape: (51171190, 26)
   • Período: 2022-01-25 00:00:00 até 2022-12-27 00:00:00
   • Features disponíveis: 26
   • Memória: 16045.8 MB
   • Estratégia: Grid Inteligente com Dask + Polars - Big Data Optimized

🔍 Metadados do processamento:
   • total_registros: 51171190
   • total_features: 26
   • combinacoes_pdv_produto: 1044310
   • semanas_cobertas: 49
   • periodo_treino: 2022-01-25 00:00:00 a 2022-12-27 00:00:00
   • estrategia: Grid Inteligente com Dask + Polars - Big Data Optimized
   • tecnologia: Dask + Polars for Maximum Performance
   • memoria_otimizada: 9974.253155708313 MB

✅ Pronto para modelagem!


## 2. Preparação dos Dados para ML

In [3]:
# Definir variável target e features
target = 'quantidade'

# Features a excluir (não devem ser usadas para predição)
exclude_features = [
    'pdv_id', 'produto_id', 'semana',  # IDs e data
    'quantidade',  # Target
    'valor', 'num_transacoes',  # Features que vazam informação do futuro
]

# Identificar features disponíveis
all_features = [col for col in dados.columns if col not in exclude_features]

print(f'🎯 Preparação dos dados:')
print(f'   • Target: {target}')
print(f'   • Features disponíveis: {len(all_features)}')
print(f'   • Features excluídas: {len(exclude_features)}')

# Verificar missing values nas features
missing_features = dados[all_features].isnull().sum()
missing_features = missing_features[missing_features > 0].sort_values(ascending=False)

if len(missing_features) > 0:
    print(f'\n⚠️ Features com valores missing:')
    for feature, count in missing_features.head(10).items():
        pct = (count / len(dados)) * 100
        print(f'   • {feature}: {count:,} ({pct:.1f}%)')
    
    print(f'\n🧠 Estratégia de Tratamento Inteligente:')
    print('   • distributor_id (categórica): NaN → -1 (venda direta)')
    print('   • Features numéricas: NaN → 0 (ausência = zero)')
    print('   • LightGBM aprenderá padrões específicos para valores -1/0')
else:
    print('\n✅ Nenhum valor missing nas features')

print(f'\n📋 Features finais para modelagem: {len(all_features)}')
print('💡 Missing values serão tratados como informação, não removidos')

🎯 Preparação dos dados:
   • Target: quantidade
   • Features disponíveis: 20
   • Features excluídas: 6

⚠️ Features com valores missing:
   • distributor_id: 45,202,572 (88.3%)

🧠 Estratégia de Tratamento Inteligente:
   • distributor_id (categórica): NaN → -1 (venda direta)
   • Features numéricas: NaN → 0 (ausência = zero)
   • LightGBM aprenderá padrões específicos para valores -1/0

📋 Features finais para modelagem: 20
💡 Missing values serão tratados como informação, não removidos


In [4]:
# PIPELINE FINAL: Otimização de Memória + Preparação dos Dados Completos
print('🧠 Preparando dados de treino com o ano de 2022 completo...')
print('🚀 Estratégia: Usar todos os dados disponíveis para o modelo final')

# PASSO 1: Inspecionar uso de memória atual
print(f'\n🔍 ANTES da otimização:')
memory_before = dados.memory_usage(deep=True).sum() / (1024**3)
print(f'💾 Memória total: {memory_before:.2f} GB')

# PASSO 2: Aplicar Downcasting Inteligente
print(f'\n🚀 Aplicando Downcasting...')

# Fazer uma cópia para otimização
dados_sorted = dados.copy()

# Otimizar colunas numéricas (inteiros e floats)
for col in dados_sorted.select_dtypes(include=[np.number]).columns:
    original_dtype = dados_sorted[col].dtype
    
    if dados_sorted[col].dtype.kind in ['i', 'u']:  # Inteiros
        dados_sorted[col] = pd.to_numeric(dados_sorted[col], downcast='integer')
    else:  # Floats
        dados_sorted[col] = pd.to_numeric(dados_sorted[col], downcast='float')
    
    new_dtype = dados_sorted[col].dtype
    if original_dtype != new_dtype:
        print(f'   • {col}: {original_dtype} → {new_dtype}')

# Otimizar colunas categóricas
for col in dados_sorted.select_dtypes(include=['object']).columns:
    if col not in ['semana']:  # Preservar datetime
        nunique = dados_sorted[col].nunique()
        total_rows = len(dados_sorted)
        if nunique / total_rows < 0.5:  # Se <50% valores únicos, usar category
            dados_sorted[col] = dados_sorted[col].astype('category')
            print(f'   • {col}: object → category')

print(f'✅ Downcasting concluído!')

# PASSO 3: Verificar resultado da otimização
memory_after = dados_sorted.memory_usage(deep=True).sum() / (1024**3)
memory_reduction = (memory_before - memory_after) / memory_before * 100
print(f'\n📊 DEPOIS da otimização:')
print(f'💾 Memória total: {memory_after:.2f} GB')
print(f'🎯 Redução: {memory_reduction:.1f}% ({memory_before-memory_after:.2f} GB economizados)')

# PASSO 4: Tratamento de missing values
print(f'\n🧠 Tratamento inteligente de missing values...')
all_features = [col for col in dados_sorted.columns if col not in ['pdv_id', 'produto_id', 'semana', 'quantidade', 'valor', 'num_transacoes']]

for col in all_features:
    missing_count = dados_sorted[col].isnull().sum()
    if missing_count > 0:
        if col == 'distributor_id':
            # Adicionar -1 ao "menu" de categorias primeiro
            if dados_sorted[col].dtype.name == 'category':
                if -1 not in dados_sorted[col].cat.categories:
                    dados_sorted[col] = dados_sorted[col].cat.add_categories([-1])
            
            dados_sorted[col] = dados_sorted[col].fillna(-1)
            print(f'   • {col}: {missing_count:,} NaN → -1 (venda direta)')
            
        elif dados_sorted[col].dtype.kind in ['i', 'u', 'f']:
            dados_sorted[col] = dados_sorted[col].fillna(0)
            print(f'   • {col}: {missing_count:,} NaN → 0 (ausência)')

# PASSO 5: Preparar dados COMPLETOS para modelagem (SEM divisão treino/validação)
print(f'\n🎯 Preparando dados completos para o modelo final...')

# Usar todos os dados de 2022 para o treino final
X_full = dados_sorted[all_features]
y_full = dados_sorted[target]

print(f'✅ Dados de treino preparados:')
print(f'   • X_full shape: {X_full.shape}')
print(f'   • y_full shape: {y_full.shape}')
print(f'   • Período: {dados_sorted["semana"].min()} até {dados_sorted["semana"].max()}')
print(f'   • Memória X_full: {X_full.memory_usage(deep=True).sum() / (1024**2):.1f} MB')

# Limpeza de memória
import gc
del dados
gc.collect()

print(f'\n🎉 SUCESSO! Dados preparados para treinamento final:')
print(f'   ✅ Dataset completo de 2022: {len(dados_sorted):,} registros')
print(f'   ✅ Features otimizadas: {len(all_features)}')
print(f'   ✅ Memória otimizada: {memory_reduction:.1f}% de redução')
print(f'   ✅ Pronto para treinar modelo final!')

🧠 Preparando dados de treino com o ano de 2022 completo...
🚀 Estratégia: Usar todos os dados disponíveis para o modelo final

🔍 ANTES da otimização:
💾 Memória total: 15.67 GB

🚀 Aplicando Downcasting...
   • quantidade: float64 → float32
   • num_transacoes: float64 → float32
   • mes_sin: float64 → float32
   • mes_cos: float64 → float32
   • quantidade_lag_1: float64 → float32
   • quantidade_lag_2: float64 → float32
   • quantidade_lag_3: float64 → float32
   • quantidade_lag_4: float64 → float32
   • quantidade_media_4w: float64 → float32
   • quantidade_max_4w: float64 → float32
   • quantidade_min_4w: float64 → float32
   • pdv_hash: uint64 → int8
   • produto_hash: uint64 → int8
   • pdv_produto_hash: uint64 → int16
   • hist_mean: float64 → float32
   • hist_std: float64 → float32
   • hist_max: float64 → float32
   • hist_count: uint32 → int8
   • pdv_id: object → category
   • produto_id: object → category
   • distributor_id: object → category
✅ Downcasting concluído!

📊 DEP

In [5]:
# TREINAMENTO DO MODELO FINAL
print('🚀 Treinando o modelo LightGBM final com todos os dados de 2022...')
print('=' * 60)

# Parâmetros do LightGBM (baseado nos experimentos do notebook 03)
lgb_params_final = {
    'objective': 'regression_l1',  # MAE - melhor para WMAPE
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'random_state': 42,
    'n_jobs': -1
}

print(f'\n📋 Configuração do modelo final:')
for param, value in lgb_params_final.items():
    print(f'   • {param}: {value}')

# Criar o dataset LightGBM com todos os dados
print(f'\n📊 Preparando dados para treinamento...')
train_full_lgb = lgb.Dataset(X_full, label=y_full)

print(f'   • Dados shape: {X_full.shape}')
print(f'   • Features: {len(all_features)}')
print(f'   • Target: {target}')

# O número de iterações ideais foi encontrado no notebook 03 (ajustar conforme seus experimentos)
# Usar um número fixo baseado nos experimentos anteriores
best_iteration = 200  # Substitua pelo valor encontrado nos seus experimentos

print(f'\n🔄 Treinando modelo final com {best_iteration} iterações...')

# CORREÇÃO: Removido o parâmetro "verbose_eval" que causava o erro
final_model = lgb.train(
    lgb_params_final,
    train_full_lgb,
    num_boost_round=best_iteration
)

print(f'✅ Modelo final treinado com sucesso!')

# Salvar o modelo
import pickle
import os
os.makedirs('../data', exist_ok=True)

with open('../data/final_lightgbm_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)

print('💾 Modelo salvo em: ../data/final_lightgbm_model.pkl')

# Informações do modelo
print(f'\n📈 Informações do modelo final:')
print(f'   • Número de árvores: {final_model.num_trees()}')
print(f'   • Features utilizadas: {len(all_features)}')
print(f'   • Dados de treino: {len(y_full):,} registros')
print(f'   • Período de treino: Todo o ano de 2022')
print(f'   • Pronto para predições!')

🚀 Treinando o modelo LightGBM final com todos os dados de 2022...

📋 Configuração do modelo final:
   • objective: regression_l1
   • metric: mae
   • boosting_type: gbdt
   • verbosity: -1
   • random_state: 42
   • n_jobs: -1

📊 Preparando dados para treinamento...
   • Dados shape: (51171190, 20)
   • Features: 20
   • Target: quantidade

🔄 Treinando modelo final com 200 iterações...
✅ Modelo final treinado com sucesso!
💾 Modelo salvo em: ../data/final_lightgbm_model.pkl

📈 Informações do modelo final:
   • Número de árvores: 200
   • Features utilizadas: 20
   • Dados de treino: 51,171,190 registros
   • Período de treino: Todo o ano de 2022
   • Pronto para predições!


## 3. Geração de Features para Janeiro/2023

In [6]:
# GERAÇÃO DE FEATURES PARA JANEIRO/2023
print('📅 Criando grid de dados para as 5 semanas de janeiro/2023...')
print('🎯 Estratégia: Criar template para todas as combinações PDV/Produto')

# PASSO 1: Obter todas as combinações únicas de PDV/Produto do histórico
print('\n🔍 Identificando combinações únicas de PDV/Produto...')
all_combos = dados_sorted[['pdv_id', 'produto_id']].drop_duplicates()
print(f'   • Total de combinações únicas: {len(all_combos):,}')

# PASSO 2: Definir as 5 semanas de janeiro de 2023
print('\n📅 Definindo as 5 semanas de janeiro/2023...')
prediction_weeks = pd.to_datetime(['2023-01-03', '2023-01-10', '2023-01-17', '2023-01-24', '2023-01-31'])
print(f'   • Semanas: {[w.strftime("%Y-%m-%d") for w in prediction_weeks]}')

# PASSO 3: Criar o grid de teste (todas as combinações x todas as semanas)
print('\n🏗️ Construindo grid de teste...')
test_grid_list = []
for week in prediction_weeks:
    combo_week = all_combos.copy()
    combo_week['semana'] = week
    test_grid_list.append(combo_week)

test_grid = pd.concat(test_grid_list, ignore_index=True)
print(f'   • Grid shape: {test_grid.shape}')
print(f'   • Total de predições: {len(test_grid):,}')

# PASSO 4: Gerar features baseadas no histórico de 2022
print('\n🔧 Gerando features para o período de teste...')
print('⚠️ IMPORTANTE: Este é um processo complexo que deve replicar exatamente')
print('   as features do notebook 02-Feature-Engineering-Dask.ipynb')

# Para este exemplo, vamos criar features básicas
# Em um cenário real, você precisaria replicar TODAS as features do notebook 02

# Merge com informações básicas dos PDVs e produtos
pdvs_df = pd.read_parquet('../data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet')
produtos_df = pd.read_parquet('../data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet')

# Adicionar informações básicas
test_grid = test_grid.merge(pdvs_df[['pdv_id', 'uf', 'distributor_id']], on='pdv_id', how='left')
test_grid = test_grid.merge(produtos_df[['produto_id', 'brand']], on='produto_id', how='left')

print(f'   • Features básicas adicionadas')
print(f'   • Grid com info básica: {test_grid.shape}')

# PASSO 5: Adicionar features temporais
test_grid['ano'] = test_grid['semana'].dt.year
test_grid['mes'] = test_grid['semana'].dt.month  
test_grid['semana_ano'] = test_grid['semana'].dt.isocalendar().week
test_grid['dia_ano'] = test_grid['semana'].dt.dayofyear

print(f'   • Features temporais adicionadas')

# PASSO 6: Calcular lags e rolling windows (simplificado)
print('\n📊 Calculando features de lag e rolling windows...')

# Para cada combinação PDV/Produto, calcular estatísticas do histórico
historical_stats = dados_sorted.groupby(['pdv_id', 'produto_id'])['quantidade'].agg([
    'mean', 'std', 'min', 'max', 'count'
]).reset_index()
historical_stats.columns = ['pdv_id', 'produto_id', 'hist_mean', 'hist_std', 'hist_min', 'hist_max', 'hist_count']

# Merge com estatísticas históricas
test_grid = test_grid.merge(historical_stats, on=['pdv_id', 'produto_id'], how='left')

# Para os lags, vamos usar a última semana de 2022 como "lag_1"
last_week_2022 = dados_sorted[dados_sorted['semana'] == dados_sorted['semana'].max()]
last_week_stats = last_week_2022[['pdv_id', 'produto_id', 'quantidade']].rename(columns={'quantidade': 'quantidade_lag_1'})
test_grid = test_grid.merge(last_week_stats, on=['pdv_id', 'produto_id'], how='left')

print(f'   • Features de lag e estatísticas históricas adicionadas')

# PASSO 7: Preencher valores ausentes e finalizar
test_grid['distributor_id'] = test_grid['distributor_id'].fillna(-1)
for col in ['hist_mean', 'hist_std', 'hist_min', 'hist_max', 'hist_count', 'quantidade_lag_1']:
    test_grid[col] = test_grid[col].fillna(0)

print(f'\n✅ Features para janeiro/2023 geradas com sucesso!')
print(f'   • Grid final: {test_grid.shape}')
print(f'   • Features disponíveis: {list(test_grid.columns)}')
print(f'   • Pronto para predições!')

📅 Criando grid de dados para as 5 semanas de janeiro/2023...
🎯 Estratégia: Criar template para todas as combinações PDV/Produto

🔍 Identificando combinações únicas de PDV/Produto...
   • Total de combinações únicas: 1,044,310

📅 Definindo as 5 semanas de janeiro/2023...
   • Semanas: ['2023-01-03', '2023-01-10', '2023-01-17', '2023-01-24', '2023-01-31']

🏗️ Construindo grid de teste...
   • Grid shape: (5221550, 3)
   • Total de predições: 5,221,550

🔧 Gerando features para o período de teste...
⚠️ IMPORTANTE: Este é um processo complexo que deve replicar exatamente
   as features do notebook 02-Feature-Engineering-Dask.ipynb


KeyError: "None of [Index(['pdv_id', 'uf', 'distributor_id'], dtype='object')] are in the [columns]"

## 4. Predições e Geração dos Arquivos de Submissão

In [ ]:
# PREDIÇÕES FINAIS E GERAÇÃO DE SUBMISSÃO
print('🚀 Realizando predições para janeiro/2023...')
print('🎯 Usando modelo final treinado com todo o dataset de 2022')

# PASSO 1: Preparar dados de teste com as mesmas features do treino
print('\n🔧 Preparando dados de teste...')

# Identificar features que existem tanto no treino quanto no teste
available_test_features = [col for col in all_features if col in test_grid.columns]
missing_features = [col for col in all_features if col not in test_grid.columns]

print(f'   • Features disponíveis no teste: {len(available_test_features)}')
if missing_features:
    print(f'   ⚠️ Features ausentes no teste: {len(missing_features)}')
    print(f'     Criando com valores padrão: {missing_features[:5]}...')
    
    # Criar features ausentes com valores padrão (0 ou -1)
    for feature in missing_features:
        if 'distributor' in feature or 'id' in feature:
            test_grid[feature] = -1
        else:
            test_grid[feature] = 0

# Selecionar apenas as features que foram usadas no treinamento
X_test = test_grid[all_features]

print(f'   • X_test shape: {X_test.shape}')
print(f'   • Features utilizadas: {len(all_features)}')

# PASSO 2: Aplicar o mesmo tratamento de tipos de dados
print('\n🔧 Aplicando otimizações de tipos de dados...')
for col in X_test.select_dtypes(include=[np.number]).columns:
    if X_test[col].dtype.kind in ['i', 'u']:  # Inteiros
        X_test[col] = pd.to_numeric(X_test[col], downcast='integer')
    else:  # Floats
        X_test[col] = pd.to_numeric(X_test[col], downcast='float')

# PASSO 3: Gerar predições
print('\n🎯 Gerando predições...')
predictions = final_model.predict(X_test)

# Garantir que as previsões não sejam negativas e arredondar para inteiros
predictions = np.maximum(0, predictions).round().astype(int)

print(f'   • Predições geradas: {len(predictions):,}')
print(f'   • Range: {predictions.min()} - {predictions.max()}')
print(f'   • Média: {predictions.mean():.2f}')
print(f'   • Zeros: {(predictions == 0).sum():,} ({(predictions == 0).mean()*100:.1f}%)')

# PASSO 4: Criar o DataFrame de submissão
print('\n📋 Criando DataFrame de submissão...')

# Preparar dados para submissão
submission_df = test_grid[['semana', 'pdv_id', 'produto_id']].copy()
submission_df['quantidade'] = predictions

# Converter semana para formato numérico (semana 1-5 de janeiro)
submission_df['semana_num'] = submission_df['semana'].dt.isocalendar().week
# Como são as semanas 1-5 de janeiro de 2023, ajustar numeração
primeira_semana = submission_df['semana_num'].min()
submission_df['semana'] = submission_df['semana_num'] - primeira_semana + 1

# Renomear colunas para o padrão da submissão
submission_df = submission_df.rename(columns={
    'semana': 'semana',
    'pdv_id': 'pdv', 
    'produto_id': 'produto'
})

# Selecionar apenas as colunas finais
submission_df = submission_df[['semana', 'pdv', 'produto', 'quantidade']]

print(f'   • DataFrame de submissão criado: {submission_df.shape}')
print(f'   • Colunas: {list(submission_df.columns)}')
print(f'   • Semanas: {sorted(submission_df["semana"].unique())}')

print(f'\\n📊 Estatísticas da submissão:')
for semana in sorted(submission_df['semana'].unique()):
    week_data = submission_df[submission_df['semana'] == semana]['quantidade']
    print(f'   • Semana {semana}: {len(week_data):,} predições, média: {week_data.mean():.1f}')

print(f'\\n✅ Dados de submissão preparados!')

In [ ]:
# SALVAMENTO DOS ARQUIVOS DE SUBMISSÃO
print('💾 Salvando arquivos de submissão...')
print('🎯 Formatos: CSV (com separador ;) e Parquet')

# PASSO 1: Criar pasta de submissão
output_dir = '../submissions'
os.makedirs(output_dir, exist_ok=True)
print(f'   • Pasta criada: {output_dir}')

# PASSO 2: Validar DataFrame antes de salvar
print(f'\n🔍 Validação final do DataFrame:')
print(f'   • Shape: {submission_df.shape}')
print(f'   • Colunas: {list(submission_df.columns)}')
print(f'   • Tipos: {submission_df.dtypes.to_dict()}')
print(f'   • Missing values: {submission_df.isnull().sum().sum()}')
print(f'   • Valores negativos: {(submission_df["quantidade"] < 0).sum()}')

# Verificar se há problemas
if submission_df.isnull().sum().sum() > 0:
    print('   ⚠️ Há valores missing - preenchendo com 0')
    submission_df = submission_df.fillna(0)

if (submission_df["quantidade"] < 0).sum() > 0:
    print('   ⚠️ Há valores negativos - convertendo para 0')
    submission_df["quantidade"] = submission_df["quantidade"].clip(lower=0)

# PASSO 3: Salvar no formato CSV
csv_path = os.path.join(output_dir, 'submission.csv')
submission_df.to_csv(
    csv_path,
    sep=';',  # Separador conforme especificação
    index=False,
    encoding='utf-8'
)
print(f'   • Arquivo CSV salvo: {csv_path}')

# PASSO 4: Salvar no formato Parquet
parquet_path = os.path.join(output_dir, 'submission.parquet')
submission_df.to_parquet(parquet_path, index=False)
print(f'   • Arquivo Parquet salvo: {parquet_path}')

# PASSO 5: Verificar arquivos salvos
import os
csv_size = os.path.getsize(csv_path) / (1024**2)  # MB
parquet_size = os.path.getsize(parquet_path) / (1024**2)  # MB

print(f'\n📁 Arquivos de submissão gerados:')
print(f'   • CSV: {csv_path} ({csv_size:.1f} MB)')
print(f'   • Parquet: {parquet_path} ({parquet_size:.1f} MB)')

# PASSO 6: Visualizar algumas linhas do resultado
print(f'\n👀 Preview da submissão (primeiras 10 linhas):')
print(submission_df.head(10).to_string(index=False))

print(f'\n🎉 PROCESSO FINALIZADO!')
print(f'=' * 60)
print(f'✅ Modelo LightGBM treinado com {len(y_full):,} registros de 2022')
print(f'✅ Predições geradas para {len(submission_df):,} combinações')
print(f'✅ Arquivos de submissão salvos em: {output_dir}')
print(f'✅ Formatos: CSV (;) e Parquet')
print(f'')
print(f'🚀 PRÓXIMOS PASSOS:')
print(f'   1. Validar os arquivos gerados')
print(f'   2. Fazer upload conforme regulamento do hackathon')
print(f'   3. Acompanhar resultados na plataforma')
print(f'')
print(f'🏆 Boa sorte no hackathon!')